# Kaggle Titanic
## Logistic Regression with Python


For this lecture we will be working with the [Titanic Data Set from Kaggle](https://www.kaggle.com/c/titanic). This is a very famous dataset.



# Step - 0

## Import Libraries

In [1]:
!pip install missingno

You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as ms
%matplotlib inline

# Step -1

Load the dataset.

Pandas provides two important data types with in built functions to be able to provide extensive capability to handle the data.The datatypes include Series and DataFrames.

# Series

A Series is very similar to a one dimensional array.The difference between an array and a Series, is that a Series can have axis labels, meaning it can be indexed by a label, instead of just a number location and it is not necessary that the index should be integer value,it can hold any arbitrary python object.

In [4]:
#this creates a series with indexing based on the position of element in list
my_list = [10,20,30]
series_1=pd.Series(my_list)
series_1

0    10
1    20
2    30
dtype: int64

In [5]:
#this example illustrates how to add index to the rows
labels=['a','b','c']
series_2=pd.Series(my_list,labels)
series_2

a    10
b    20
c    30
dtype: int64

In [6]:
#it is not necessary to specify the data and labels in the same order.the order can be changed but
#care should be taken that attributes are specified while referring to their value.
series_3=pd.Series(index=labels,data=my_list)
series_3

a    10
b    20
c    30
dtype: int64

In [7]:
#the series can be created from dictionary as
d={'a':10,'b':20,'c':30}
series_4=pd.Series(d)
series_4

a    10
b    20
c    30
dtype: int64

# DataFrames

DataFrame is another important data type of pandas.It can be thought of as a collection of series.

In [8]:
from numpy.random import randn
np.random.seed(101)

In [9]:
df = pd.DataFrame(randn(5,4),index='A B C D E'.split(),columns='W X Y Z'.split())
df

,W,X,Y,Z
A,2.706850,0.628133,0.907969,0.503826
B,0.651118,-0.319318,-0.848077,0.605965
C,-2.018168,0.740122,0.528813,-0.589001
D,0.188695,-0.758872,-0.933237,0.955057
E,0.190794,1.978757,2.605967,0.683509


Pandas provides ways to read or get the data from various sources like read_csv,read_excel,read_html etc.The data is read and stored in the form of DataFrames.

In [ ]:
data = pd.read_csv('titanic.csv')
data.head() #this reads the first five entries in the data read from the source

In [ ]:
#to get the last 5 entries of the data
data.tail()

In [ ]:
data.shape

In [ ]:
data.describe()

In [ ]:
data.info()

### Accessing individual data in the data frame

#### Working with Columns

since each dataframe is a collection of series if we access a single column we get a series object

In [ ]:
s=data['Cabin'].head()

In [ ]:
type(s)

In [ ]:
data[['Cabin','Parch']].head()

In [ ]:
data.Cabin.head()

In [ ]:
#to add a new column which is similar to parch(for illustration)
data['New_parch']=data['Parch']
data['New_parch'].head()

In [ ]:
data.info() #New_parch is added to the set of columns in the data frames

### Working with rows

There are two ways to access the rows of a data frame.It can be done through indexing or by using the name of the index 

In [ ]:
data.iloc[2] #this is to access second row of the data frame using index

In [ ]:
data.loc[67] #since here indexing is based on numbers

In [ ]:
#or taking the example data frame created during illustration i.e
df

In [ ]:
df.loc['A']

To drop columns and rows we use drop method

In [ ]:
data.drop('New_parch',axis=1).head() #axis=1 specifies we are dealing with columns

In [ ]:
#the above command does not modify the original dataframe.To modify the original dataframe we use
data.drop('New_parch',axis=1,inplace=True)

In [ ]:
#we perform conditional based accessing on the dataframe as
data[data['Pclass']>2].head()#return rows which satisfy the condition

### Merging

In [ ]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                     'A': ['A0', 'A1', 'A2', 'A3'],
                     'B': ['B0', 'B1', 'B2', 'B3']})
   
right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],
                          'C': ['C0', 'C1', 'C2', 'C3'],
                          'D': ['D0', 'D1', 'D2', 'D3']})  

In [ ]:
pd.merge(left,right,how='inner',on='key')

# Step - 2

Exploratory Data - Analysis


Visualizing missing values in the dataset.

In [ ]:
ms.matrix(data)

In [ ]:
data['Cabin'].value_counts()

We can observe that there are missing values in 'Age', 'Cabin' and 'Embarked'. Let's continue.

In [ ]:
data.info()

Visualization of data with Seaborn

In [ ]:
sns.jointplot(x='Fare',y='Age',data=data)

In [ ]:
sns.distplot(data['Fare'])

In [ ]:
sns.heatmap(data.corr(),cmap='coolwarm')
plt.title('data.corr()')

In [ ]:
sns.swarmplot(x='Pclass',y='Age',data=data,palette='Set2')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',data=data,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Sex',data=data,palette='RdBu_r')

In [ ]:
sns.set_style('whitegrid')
sns.countplot(x='Survived',hue='Pclass',data = data,palette='rainbow')

In [ ]:
data['Age'].hist(bins = 30, color = 'darkred', alpha = 0.8)

In [ ]:
sns.countplot(x = 'SibSp', data = data)

In [ ]:
data['Fare'].hist(color = 'green', bins = 40, figsize = (8,3))

#### What do you observe from the above charts?

# Step - 3


## Data Cleaning

We want to fill the missing values of the age in the dataset with the average age value for each of the classes. This is called data imputation.

In [ ]:
plt.figure(figsize=(12, 7))
sns.boxplot(x='Pclass',y='Age',data=data,palette='winter')

The average age for each of the classes are estimated to be as follows:
  
  * For **Class 1** - The average age is 37
  * For **Class 2** - The average age is 29
  * For **Class 3** - The average age is 24
  
Let's impute these values into the age column.

In [ ]:
def impute_age(cols):
    Age = cols[0]
    Pclass = cols[1]
    
    if pd.isnull(Age):
        # Class-1
        if Pclass == 1:
            return 37
        # Class-2 
        elif Pclass == 2:
            return 29
        # Class-3
        else:
            return 24

    else:
        return Age

Applying the function.

In [ ]:
data['Age'] = data[['Age','Pclass']].apply(impute_age,axis=1)

Now let's visualize the missing values.

In [ ]:
ms.matrix(data)

The Age column is imputed sucessfully.

Let's drop the Cabin column and the row in the Embarked that is NaN.

In [ ]:
data.drop('Cabin', axis = 1,inplace=True)

In [ ]:
data.head()

In [ ]:
data.dropna(inplace = True)

In [ ]:
ms.matrix(data)

## Converting Categorical Features 

We'll need to convert categorical features to dummy variables using pandas! Otherwise our machine learning algorithm won't be able to directly take in those features as inputs.

In [ ]:
data.info()

In [ ]:
sex = pd.get_dummies(data['Sex'],drop_first=True)
embark = pd.get_dummies(data['Embarked'],drop_first=True)
sex


In [ ]:
embark

In [ ]:
data.drop(['Sex','Embarked','Name','Ticket'],axis=1,inplace=True)
data.head()


In [ ]:
data = pd.concat([data,sex,embark],axis=1)

In [ ]:
data.head()

# Step - 4

## Building a Logistic Regression model


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data.drop('Survived',axis=1), 
                                                    data['Survived'], test_size=0.30, 
                                                    random_state=101)

In [ ]:
from sklearn.linear_model import LogisticRegression

# Build the Model.
logmodel = LogisticRegression()
logmodel.fit(X_train,y_train)

In [ ]:
predict =  logmodel.predict(X_test)
predict

Let's move on to evaluate our model.

## Evaluation

We can check precision, recall, f1 - score using classification report!

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report

In [ ]:
print(confusion_matrix(y_test, predict))

---
                                                     THE END